## OHLC Stock Data
Gather OHLC (open, high, low, close) data for selected stocks from Yahoo Finance. The following code provides two methods of data collection.
1. Pull the full OHLC data for all available timeframes for each stock symbol in a list.
2. Pull OHLC data for specified dates for each stock symbol in a list. (Useful for appending existing SQL database.)

In [1]:
import pandas as pd
import numpy as np
from datetime import date
import yfinance as yf

from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2

# connect to Docker SQL database
#engine = create_engine('postgresql+psycopg2://postgres:########@###.###.##.##:54320/finance')

#### Get Complete OHLC Data
Using the S&P 500 (pulled from existing SQL database), collect OHLC data for each stock symbol in a list and write data to SQL database.

Function takes any list of equity symbols, so these could be a list of symbols in an ETF or companies that you own in a portfolio.

In [2]:
# grab the sp500
stocks = pd.read_sql("SELECT DISTINCT symbol FROM equity_info.sp500", con = engine)
symbols = list(stocks['symbol'].sort_values())

# check contents
symbols[1:10]

['AAL', 'AAP', 'AAPL', 'ABBV', 'ABC', 'ABMD', 'ABT', 'ACN', 'ADBE']

In [ ]:
import time
for i in symbols[0:len(symbols)]:
    
    try:
        ohlc = yf.Ticker(i).history(period="max")
        ohlc.to_sql(name = i, schema = 'ohlc_yf', con=engine, if_exists='append')
        time.sleep(.5)
    
    except:
        print('Error obtaining OHLC for ' + str(i))
        pass

#### Update OHLC Data
After running the code above, for the future only new data need to be collected and added to the database. The following function gathers a list of all tables in a SQL database of OHLC data to get a list of equities for which data already exist. Then, a 

In [ ]:
# what do we have data for already in database? - list all tables in schema
db_tables = pd.read_sql("SELECT table_name FROM information_schema.tables \
       WHERE table_schema = 'ohlc_yf'", con = engine)
tables = db_tables['table_name'].tolist()
tables[0:10]

In [ ]:
# define variables for use in functions below
last_dates = list()
tickers_list = list()
today = list()

# list all tables in schema of OHLC data
db_tables = pd.read_sql("SELECT table_name FROM information_schema.tables \
       WHERE table_schema = 'ohlc_yf'", con = engine)
tables = db_tables['table_name'].tolist()

# turn list of tables into a dataframe of stock symbol, last date of data, today's date
for i in tables:
    
    try:
        ticker_df = pd.read_sql_table(i, engine, schema='ohlc')
        
        last_date = ticker_df['Date'].max()
        ticker_symbol = i
        
        last_dates.append(last_date)
        tickers_list.append(ticker_symbol)
        today.append(date.today())
    
    except:
        print('Error obtaining data for ' + str(i))
        pass


# append these lists into a dataframe
df = pd.DataFrame(list(zip(tickers_list, last_dates, today)),
                 columns = ['symbol','last_date','today'])

df

In [ ]:
# for each ticker, grab data for the missing dates up to today and paste them to sql database
for a, b, c in df.itertuples(index=False):
    
    try:
        ohlc = yf.Ticker(a).history(start = b, end = c)
        ohlc.to_sql(name = a, schema = 'ohlc_yf', con=engine, if_exists='append')
        time.sleep(.5)
    
    except:
        print('Error obtaining OHLC for ' + str(a))
        pass

In [ ]:
# read SQL data into python to verify function worked using first symbol in the dataframe
pd.read_sql_table(table_name = df.iloc[0,0], schema='ohlc_yf', con=engine)